In [56]:
import pandas as pd
import openpyxl

In [57]:
year = '2018'

In [58]:
df_follow = pd.read_csv(rf"C:\Users\PC\Daniel Chen Dropbox\Alexis Malca\Peru_Justice\02_Data\08_CEJ_Web\data_cleaned\DF_follow_up_cleaner_{year}.csv", error_bad_lines=False)
df_follow = df_follow.dropna(subset=['link'])
df_file = pd.read_csv(rf"C:\Users\PC\Daniel Chen Dropbox\Alexis Malca\Peru_Justice\02_Data\08_CEJ_Web\data_cleaned\DF_file_report_{year}.csv")
df_file = df_file.drop_duplicates()

C:\Users\PC\AppData\Local\Temp\ipykernel_12604\2031918126.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_follow = pd.read_csv(rf"C:\Users\PC\Daniel Chen Dropbox\Alexis Malca\Peru_Justice\02_Data\08_CEJ_Web\data_cleaned\DF_follow_up_cleaner_{year}.csv", error_bad_lines=False)
Skipping line 147793: expected 10 fields, saw 18

Skipping line 403502: expected 10 fields, saw 14
Skipping line 403503: expected 10 fields, saw 13
Skipping line 406865: expected 10 fields, saw 19
Skipping line 411463: expected 10 fields, saw 19
Skipping line 417503: expected 10 fields, saw 21
Skipping line 424997: expected 10 fields, saw 16
Skipping line 426098: expected 10 fields, saw 11
Skipping line 426296: expected 10 fields, saw 12
Skipping line 432261: expected 10 fields, saw 13
Skipping line 433084: expected 10 fields, saw 12
Skipping line 436799: expected 10 fields, saw 16

Skipping line 556892: expected 

In [59]:
# Step 1: Identify duplicated 'Expediente:' values
duplicated_expedientes = df_file[df_file.duplicated(subset=['Expediente N°:'], keep=False)]
# Step 2: Identify rows with '<NO DEFINIDO>' in 'Juez:'
no_definido_juez_rows = df_file[df_file['Juez:'] == '<NO DEFINIDO>']
# Step 3: Combine conditions to identify rows to drop
rows_to_drop = duplicated_expedientes[duplicated_expedientes['Juez:'] == '<NO DEFINIDO>'].index
# Step 4: Drop identified rows from the original DataFrame
df_file = df_file.drop(index=rows_to_drop)

In [60]:
columns_to_include = ['Expediente N°:', 'Órgano Jurisdiccional:', 'Distrito Judicial:', 'Juez:', 'Fecha de Inicio:',
                      'Proceso:','Observación:','Especialidad:','Materia(s):'] 

# Select only the specified columns from df_follow
df_file = df_file[columns_to_include]

In [61]:
step_1 = pd.merge(df_follow, df_file, on='Expediente N°:', how='left')
step_1 = step_1.drop_duplicates()

In [62]:
df_downloads = pd.read_csv(rf"C:\Users\PC\Daniel Chen Dropbox\Alexis Malca\Peru_Justice\02_Data\08_CEJ_Web\data_cleaned\DF_DOWNLOADS_{year}.csv")
df_downloads = df_downloads.drop_duplicates(subset=['link'])

In [63]:
step_2 = pd.merge(step_1, df_downloads, on='link', how='left')

columns_to_remove = [
    'Tipo de Notificación:',
    'Fojas/Folios:',
    'Descripción de Usuario:',
    'expediente_num',
    'num',
    'error',
    'file_path',
    'Observación:'
]

# Remove the specified columns
step_2 = step_2.drop(columns=columns_to_remove)

In [64]:
new_columns = {
    'Expediente N°:': 'case_file',
    'link': 'id',
    'Fecha de Resolución/Ingreso:': 'issued_date',
    'Resolución:': 'resolution_number',
    'Acto:': 'resolution_type',
    'Proveido:': 'notified_date',
    'Sumilla:': 'summary',
    'Órgano Jurisdiccional:': 'court',
    'Distrito Judicial:': 'judicial_district',
    'Juez:': 'judge',
    'Especialista Legal:': 'clerk',
    'Fecha de Inicio:': 'start_date',
    'Proceso:': 'procedure_type',
    'Especialidad:': 'law_field',
    'Materia(s):': 'sub_law_field',
    'Estado:': 'status',
    'Etapa Procesal:': 'procedural_stage',
    'text': 'opinion_text'
}

# Rename the columns
step_2 = step_2.rename(columns=new_columns)

In [65]:
from datetime import datetime

step_2['start_date'] = step_2['start_date'].str.strip()

# Convert 'start_date' column to datetime format
step_2['start_date'] = pd.to_datetime(step_2['start_date'], format='%d/%m/%Y', errors='coerce')

# Convert 'issued_date' column to datetime format
step_2['issued_date'] = pd.to_datetime(step_2['issued_date'], errors='coerce')

C:\Users\PC\AppData\Local\Temp\ipykernel_12604\1359599876.py:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  step_2['issued_date'] = pd.to_datetime(step_2['issued_date'], errors='coerce')


In [66]:
import regex as re 

# # Define patterns
pattern1 = r'J\s*U\s*E\s*Z\s*:\s*'
pattern3 = r'\n'

# Extract text using regex
def extract_judge(row):
    opinion_text = row['opinion_text']
    
    # Ensure the opinion_text is a string
    if not isinstance(opinion_text, str):
        return None
    
    match1 = re.search(pattern1, opinion_text, flags=re.IGNORECASE | re.DOTALL)

    if match1:
        start_index = match1.end()
        # Find the next newline after the start_index
        match3 = re.search(pattern3, opinion_text[start_index:], flags=re.IGNORECASE | re.DOTALL)
        
        if match3:
            end_index = start_index + match3.start()
        else:
            end_index = len(opinion_text)

        return opinion_text[start_index:end_index].strip()
    else:
        return None

In [67]:
step_2['judge_from_opinion'] = step_2.apply(extract_judge, axis = 1)

In [68]:
step_2.to_csv(rf"D:\Proyectos\amag\case_files_{year}.csv", index=False)